# Reventar a las casas de apuestas

### Imports and options

TODO:
 - Change Bwin page to https://sports.bwin.es/es/sports/tenis-5/apuestas
 - Links in bets
 - Houses in bets

In [95]:
import sqlite3
import time
from bs4 import BeautifulSoup
from abc import ABC
import Levenshtein as lev
import string
from collections import defaultdict

In [2]:
# Source: https://github.com/JeffSackmann/tennis_atp y https://github.com/JeffSackmann/tennis_wta
"""
atp, wta, all = set(), set(), set()
for row in open("atp_players.csv"):
    player = row.split(',')
#    if player[4] == "" or int(player[4][:4]) <= 1980:
#        continue
    atp.add((player[1] + ' ' + player[2]).upper())
for row in open("wta_players.csv"):
    player = row.split(',')
#    if player[4] == "" or int(player[4][:4]) <= 1980:
#        continue
    wta.add((player[1] + ' ' + player[2]).upper())
all_players = atp.union(wta)
print(f"ATP total: {len(atp)}")
print(f"WTA total: {len(wta)}")
print(f"ALL total: {len(all_players)}")
"""

'\natp, wta, all = set(), set(), set()\nfor row in open("atp_players.csv"):\n    player = row.split(\',\')\n#    if player[4] == "" or int(player[4][:4]) <= 1980:\n#        continue\n    atp.add((player[1] + \' \' + player[2]).upper())\nfor row in open("wta_players.csv"):\n    player = row.split(\',\')\n#    if player[4] == "" or int(player[4][:4]) <= 1980:\n#        continue\n    wta.add((player[1] + \' \' + player[2]).upper())\nall_players = atp.union(wta)\nprint(f"ATP total: {len(atp)}")\nprint(f"WTA total: {len(wta)}")\nprint(f"ALL total: {len(all_players)}")\n'

In [3]:
conn = sqlite3.connect("bets_database.db")
cursor = conn.cursor()

#If the table does not exists, we create it
if cursor.execute("PRAGMA table_info(TENNIS_PLAYERS)").fetchall() == []:
    tennis_players_table_query = """CREATE TABLE TENNIS_PLAYERS (
        name TEXT     UNIQUE,
        sex  TEXT (1) 
    );"""
    cursor.execute(tennis_players_table_query)
    conn.commit()

In [4]:
all_players = set([x[0].upper() for x in cursor.execute("SELECT name FROM TENNIS_PLAYERS").fetchall()])

In [90]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC # Wait for condition

options = webdriver.ChromeOptions()
options.add_argument('ignore-certificate-errors')
options.add_argument("--window-size=1920,1080")
#options.add_argument('--headless')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

#options.add_experimental_option('excludeSwitches', ['enable-logging'])

driver = webdriver.Chrome(options=options)

In [60]:
class Bet():
    def __init__(self, teams, bets):
        self.teams = [x.strip().upper() for x in teams]
        self.real_teams = []
        self.bets = [float(x.replace(',', '.')) for x in bets]
        self.date_time = None
        self.link = None
        self.house = None
    
    def is_error(self):
        if len(self.real_teams) < 2:
            return True
        return False

    def __repr__(self):
        if self.is_error():
            return f"ERROR BET ({self.teams})"
        return f"{self.real_teams[0]} ({self.bets[0]}) x {self.real_teams[1]} ({self.bets[1]})"
    
    def __eq__(self, x):
        if self.is_error() or x.is_error():
            return False
        return self.real_teams == x.real_teams
    
    def hash(self):
        if self.is_error():
            return None
        return f"{self.real_teams[0]}{self.real_teams[1]}".replace(' ', '')

In [7]:
class House(ABC):
    def link_sport(self, sport):
        pass
    def sport_bets(self, sport):
        pass
    def real_names(self):
        pass

In [112]:
class Retabet(House):
    def __init__(self):
        self.link = "https://apuestas.retabet.es"
        self.sports = {"tennis" : "tenis-m8"}
        self.ret_bets = []

    def link_sport(self, sport):
        return f"{self.link}/deportes/{self.sports[sport]}"

    def sport_bets(self, sport):
        self.ret_bets = []
        driver.get(self.link_sport(sport))
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        table = soup.find('ul', attrs={'class':'module__events-cont jbgroup'}) #TODO findAll
        for row in table:
            row = row.find('a', attrs={'class':'jlink'})
            teams = row.find('div', attrs={'class':'module__event-info'})
            teams = teams.find('ul', attrs={'class':'module__event-players'})
            teams = list(map(lambda elem: elem.text, teams.findAll('li')))

            bets = row.find('div', attrs={'class':'list-events__element__bets'})
            bets = row.find('div', attrs={'class':'module__bets-list jbet'})
            bets = row.findAll('li', attrs={'class':'jo module__bets-li'})
            bets_list = []
            for bet, _ in zip(bets, range(2)):
                bet = bet.find('span', attrs={'class':'jpr module__bets-cuota'})
                bets_list.append(bet.text)

            bet = Bet(teams, bets_list)
            self.ret_bets.append(bet)
        self.real_names()
        return self.ret_bets
    
    def real_names(self):
        for bet in self.ret_bets:
            for team in bet.teams:
                cnt, l_f = 0, []
                for player in all_players:
                    if player[0] == team[0] and player.split()[-1] == team.split()[-1]:
                        cnt += 1
                        l_f.append((player, lev.jaro_winkler(bet.teams[0][::-1], player[::-1])))
                l_f.sort(key=lambda x: x[1], reverse=True)
                if cnt == 1:
                    bet.real_teams.append(l_f[0][0])

In [111]:
class Bwin(House):
    def __init__(self):
        self.link = "https://sports.bwin.es/es/sports"
        self.sports = {"tennis" : "tenis-5"}
        self.ret_bets = []

    def link_sport(self, sport):
        return f"{self.link}/{self.sports[sport]}"

    def sport_bets(self, sport):
        self.ret_bets = []
        driver.get(self.link_sport(sport))
        while True:
            try:
                driver.find_element_by_css_selector('#main-view > ms-widget-layout > ms-widget-slot.col-9.widget-slot')
                break
            except:
                time.sleep(0.5)
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        table1 = soup.find('ms-widget-slot', attrs={'class':'col-9 widget-slot'})
        table2 = table1.find('ms-tabbed-grid', attrs={'class':'grid-wrapper tabbed-grid-widget'})
        table3 = table2.find('ms-grid')
        table4 = table3.find('ms-event-group', attrs={'class':'event-group'})
        rows = table4.findAll('ms-event', attrs={'class':'grid-event ms-active-highlight'})
        
        ret_bets = []
        for row in rows:
            try:
                row = row.find('div', attrs={'class':'grid-event-wrapper'})
                
                teams = row.find('a', attrs={'class':'grid-info-wrapper fixed'})
                teams = teams.find('ms-event-detail', attrs={'class':'grid-event-detail'})
                teams = teams.find('ms-event-name', attrs={'class':'grid-event-name'})
                teams = teams.find('ms-inline-tooltip')
                teams = teams.find('div', attrs={'class':'participants-pair-game'})
                teams = teams.findAll('div', {'class':['participant-wrapper ng-star-inserted', 'participant-wrapper']})
                teams_final = []
                for team in teams:
                    team = team.find('div', attrs={'class':'participant'})
                    team = team.find(text=True, recursive=False).strip()
                    teams_final.append(team)

                bets = row.find('div', attrs={'class':'grid-group-container'})
                bets = row.find('ms-option-group', attrs={'class':'grid-option-group grid-group'})
                bets = row.findAll('ms-option', attrs={'class':'grid-option'})
                bets_final = []
                for bet, _ in zip(bets, range(2)):
                    bets_final.append(bet.text)

                bet = Bet(teams_final, bets_final)
                self.ret_bets.append(bet)
            except:
                pass
        self.real_names()
        return self.ret_bets

    def real_names(self):
        for bet in self.ret_bets:
            for team in bet.teams:
                cnt, l_f = 0, []
                for player in all_players:
                    if player[0] == team[0] or player[1] == team[1]:
                        cnt += 1
                        l_f.append((player, lev.jaro_winkler(player, team)))
                l_f.sort(key=lambda x: x[1], reverse=True)
                bet.real_teams.append(l_f[0][0])

In [110]:
class WilliamHill(House):
    def __init__(self):
        self.link = "https://sports.williamhill.es/betting/es-es"
        self.sports = {"tennis" : "tenis/partidos"}
        self.ret_bets = []

    def link_sport(self, sport):
        return f"{self.link}/{self.sports[sport]}"

    def sport_bets(self, sport):
        self.ret_bets = []
        driver.get(self.link_sport(sport))
        for _ in range(3):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.2)
        response = driver.page_source
        soup = BeautifulSoup(response, "html.parser")
        matches = soup.findAll('div', attrs={'class':'event'})
        for match in matches:
            teams = match.find('a')
            teams_final, bets_final = [], []
            teams_final = teams['title'].split('₋')
            bets = match.findAll('button', attrs={'class':'btn betbutton oddsbutton'})
            for bet in bets:
                bets_final.append(bet.text)
            self.ret_bets.append(Bet(teams_final, bets_final))
        self.real_names()
        return self.ret_bets

    def real_names(self):
        for bet in self.ret_bets:
            for team in bet.teams:
                cnt, l_f = 0, []
                for player in all_players:
                    if player[0] == team[0] or player[1] == team[1]:
                        cnt += 1
                        l_f.append((player, lev.jaro_winkler(player, team)))
                l_f.sort(key=lambda x: x[1], reverse=True)
                bet.real_teams.append(l_f[0][0])

In [114]:
reta = Retabet()
bwin = Bwin()
william_hill = WilliamHill()
house_list = [reta, bwin, william_hill]

In [ ]:
william_hill.link_sport("tennis")

In [131]:
%%time
match_dict = defaultdict(lambda: [])
for house in house_list:
    house.sport_bets("tennis")
    for match in house.ret_bets:
        if match.is_error():
            continue
        match_dict[match.hash()].append(match)
match_dict

Wall time: 10.9 s


defaultdict(<function __main__.<lambda>()>,
            {'JENSONBROOKSBYPETERGOJOWCZYK': [JENSON BROOKSBY (1.7) x PETER GOJOWCZYK (2.1),
              JENSON BROOKSBY (1.78) x PETER GOJOWCZYK (2.0),
              JENSON BROOKSBY (1.73) x PETER GOJOWCZYK (2.1)],
             'FEDERICOGAIOFLAVIOCOBOLLI': [FEDERICO GAIO (2.0) x FLAVIO COBOLLI (1.72),
              FEDERICO GAIO (1.95) x FLAVIO COBOLLI (1.75),
              FEDERICO GAIO (1.83) x FLAVIO COBOLLI (1.83)],
             'JORDANTHOMPSONMAXIMECRESSY': [JORDAN THOMPSON (1.45) x MAXIME CRESSY (2.6),
              JORDAN THOMPSON (1.5) x MAXIME CRESSY (2.55),
              JORDAN THOMPSON (1.53) x MAXIME CRESSY (2.5)],
             'ALEKSANDRNEDOVYESOVTOMÁSLIPOVSEKPUCHES': [ALEKSANDR NEDOVYESOV (6.75) x TOMÁS LIPOVSEK PUCHES (1.09)],
             'FRANCESCAJONESARIANNEHARTONO': [FRANCESCA JONES (2.0) x ARIANNE HARTONO (1.72)],
             'MARYNAZANEVSKANATALIAVIKHLYANTSEVA': [MARYNA ZANEVSKA (1.55) x NATALIA VIKHLYANTSEVA (2.35)]

In [81]:
def retorno(ca, cb):
    return ca / (1 + (ca / cb))

In [132]:
for key, match in match_dict.items():
    if len(match) <= 1:
        continue
    for m1 in match:
        for m2 in match:
            ret = retorno(m1.bets[0], m2.bets[1])
            if ret >= 1:
                print(ret)
                print(m1, m2)

In [18]:
driver.get("https://m.apuestas.codere.es/deportes/index.htm#/HomePage")

In [19]:
driver.get("https://betway.es/es/sports")